# Perform Hyperparameter tuning on Churn prediciton dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
dataset = pd.read_csv("https://raw.githubusercontent.com/krishnaik06/Complete-Deep-Learning/master/ANN/Churn_Modelling.csv")
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]

In [4]:
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [5]:
X=pd.concat([X,geography,gender],axis=1)

In [6]:
X=X.drop(['Geography','Gender'],axis=1)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

C:\Users\KRUSHNA\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\KRUSHNA\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\KRUSHNA\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\KRUSHNA\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprecated and will be

# Perform Hyperparameter Optimization

In [12]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense,Activation,Embedding,Flatten,LeakyReLU,BatchNormalization,Dropout
from keras.activations import relu,sigmoid

In [16]:
def create_model(layers,activation):
    model = Sequential()
    for i,nodes in enumerate(layers):
        if i == 0:
            model.add(Dense(nodes,input_dim =X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
    
    model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy']
        )
    return model

In [26]:
X_train.shape[1]

11

In [13]:
layers = [[20],[50,46],[45,65,67]]
for i,node in enumerate(layers):
    print(i,node)

0 [20]
1 [50, 46]
2 [45, 65, 67]


In [18]:
model = KerasClassifier(build_fn = create_model,verbose=0)

C:\Users\KRUSHNA\AppData\Local\Temp\ipykernel_24852\1170972873.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn = create_model,verbose=0)


In [19]:
layers = [[20], [40, 20], [45, 30, 15]]
activations = ['sigmoid', 'relu']

In [20]:
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

In [22]:
param_grid

{'layers': [[20], [40, 20], [45, 30, 15]],
 'activation': ['sigmoid', 'relu'],
 'batch_size': [128, 256],
 'epochs': [30]}

In [23]:
grid

GridSearchCV(cv=5,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000002AD1361C3D0>,
             param_grid={'activation': ['sigmoid', 'relu'],
                         'batch_size': [128, 256], 'epochs': [30],
                         'layers': [[20], [40, 20], [45, 30, 15]]})

In [21]:
grid_result = grid.fit(X_train, y_train)

In [24]:
grid_result

GridSearchCV(cv=5,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000002AD1361C3D0>,
             param_grid={'activation': ['sigmoid', 'relu'],
                         'batch_size': [128, 256], 'epochs': [30],
                         'layers': [[20], [40, 20], [45, 30, 15]]})

In [25]:
[grid_result.best_score_,grid_result.best_params_]

[0.8536249995231628,
 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [40, 20]}]